# Clipping Pseudocode

1. An algorithm to clip a scene against a set of clipping planes (scene을 clipping)
2. An algorithm to clip an instance against a set of clipping planes (instance를 clipping)
3. An algorithm to clip a set of triangles against a clipping plane (삼각형을 clipping)
4. A function to compute the signed distance from a plane to a point (평면에서 점까지 부호를 계산하는 함수)

In [18]:
import numpy as np
import math
import cv2
import matplotlib.pyplot as plt

In [19]:
def rotation_matrix(orientation):
    x_rotate = np.array([[1, 0, 0, 0],
                   [0, math.cos(math.pi * orientation[0] / 180), (math.sin(math.pi * orientation[0] / 180)), 0],
                   [0, -(math.sin(math.pi * orientation[0] / 180)), math.cos(math.pi * orientation[0] / 180), 0],
                    [0, 0, 0, 1]])
    y_rotate = np.array([[math.cos(math.pi * orientation[1] / 180),0,-(math.sin(math.pi * orientation[1] / 180)), 0],
               [0, 1, 0, 0],
               [(math.sin(math.pi * orientation[1] / 180)), 0, math.cos(math.pi * orientation[1] / 180), 0],
                [0, 0, 0, 1]])
    z_rotate = np.array([[math.cos(math.pi * orientation[2] / 180), (math.sin(math.pi * orientation[2] / 180)), 0, 0],
               [-(math.sin(math.pi * orientation[2] / 180)), math.cos(math.pi * orientation[2] / 180), 0, 0],
               [0, 0, 1, 0],
                [0, 0, 0, 1]])
    
    return np.dot(np.dot(x_rotate, y_rotate), z_rotate)

def translate_matrix(position):
    return np.array([[1, 0, 0, position[0]],
                   [0, 1, 0, position[1]],
                   [0, 0, 1, position[2]],
                   [0, 0, 0, 1]])

def scale_matrix(scale):
    return np.array([[scale, 0, 0, 0],
                    [0, scale, 0, 0],
                    [0, 0, scale, 0],
                    [0, 0, 0, 1]])

In [24]:
class Point:
    def __init__(self, x, y, h, color=[255,255,255]):
        self.x = x
        self.y = y
        self.h = h
        self.color = color
        
class Vertex:
    def __init__(self, x, y, z):
        self.x = x
        self.y = y
        self.z = z
        
class Vertex4:
    def __init__(self, x, y, z, w):
        self.x = x
        self.y = y
        self.z = z
        self.w = w

class Triangle:
    def __init__(self, v1,v2,v3,color):
        self.v = [v1,v2,v3]
        self.color = color
        
class Camera:
    def __init__(self,
                 position = [0,0,0],
                 orientation = [0,0,0],
                 viewport_size = 1,
                 projection_plane_z = 1
                 ):
        
        self.position = position
        self.orientation = orientation
        self.viewport_size = viewport_size
        self.projection_plane_z = projection_plane_z
        
        plane_1 = Plane(np.array([0,0,1]),0.1)
        plane_2 = Plane(np.array([1/math.sqrt(2),0 ,1/math.sqrt(2)]),0)
        plane_3 = Plane(np.array([-1/math.sqrt(2),0 ,1/math.sqrt(2)]),0)
        plane_4 = Plane(np.array([0,1/math.sqrt(2),1/math.sqrt(2)]),0)
        plane_5 = Plane(np.array([0,-1/math.sqrt(2),1/math.sqrt(2)]),0)
        self.planes = [plane_1,plane_2,plane_3,plane_4,plane_5]
        
class Plane:
    def __init__(self, normal, d):
        self.normal = normal
        self.d = d
        
        
        
class Model:
    def __init__(self, vertices, triangles):
        self.vertices = vertices
        self.triangles = triangles
        
class Instance:
    def __init__(self, model, position=[0,0,0], orientation=[0,0,0], scale=1):
        self.model = model
        self.position = position
        self.orientation = orientation
        self.scale = scale
        #self.transform = np.dot(translate_matrix(self.position), np.dot(rotation_matrix(self.orientation), scale_matrix(self.scale)))

class Scene:
    def __init__(self):
        self.instances = []

In [30]:
class Renderer:
    def __init__(self, camera, canvas):
        self.camera = camera
        self.canvas = canvas
        
    def draw_line(self, p0, p1, color, line_width=1):
        if np.abs(p0.x - p1.x) > abs(p0.y - p1.y):
            # line is horizontal-ish
            if p0.x > p1.x:
                p0, p1 = p1, p0
            
            ys = self.interpolate(p0.x,p0.y,p1.x,p1.y)

            for i in range(p1.x-p0.x+1):
                canvas.put_pixel(i+p0.x,ys[i],color,line_width)
                    
        else:
            # line is vertical-ish
                        # line is horizontal-ish
            if p0.y > p1.y:
                p0, p1 = p1, p0
            
            xs = self.interpolate(p0.y,p0.x,p1.y,p1.x)
            for i in range(p1.y-p0.y+1):
                canvas.put_pixel(xs[i], i+p0.y, color, line_width)
                
    
    def interpolate(self, i0, d0,i1, d1):
        if i0==i1:
            return [d0]
        else:
            values = []
            
            a = (d1-d0) / (i1 - i0)
            d = d0
            
            for i in range(i0, i1+1):
                values.append(d)
                d += a
            return values
        
    def draw_wire_triangle(self, p0, p1, p2, color, line_width = 1):
        self.draw_line(p0, p1, color, line_width)
        self.draw_line(p1, p2, color, line_width)
        self.draw_line(p0, p2, color, line_width)
        
    def draw_filled_triangle(self, p0, p1, p2, color, line_width = 1):
        # Sort the points so that y0 <= y1 <= y2
        if p1.y < p0.y:
            p1, p0 = p0, p1
        if p2.y < p0.y:
            p2, p0 = p0, p2
        if p2.y < p1.y:
            p1, p2 = p2, p1
        
        # Compute the x coordinates of the triangle edges
        x01 = self.interpolate(p0.y, p0.x, p1.y, p1.x)
        x12 = self.interpolate(p1.y, p1.x, p2.y, p2.x)
        x02 = self.interpolate(p0.y, p0.x, p2.y, p2.x)
        
        # Concatenate the short sides
        x01.pop(-1)
        x012 = x01 + x12
        
        # Determine which is left and which is right
        m = math.floor(len(x012)/2)
        if x02[m] < x012[m]:
            x_left = x02
            x_right= x012
        else:
            x_left = x012
            x_right = x02
            
        # Draw the horizontal segments
        for y in range(p0.y,p2.y):
            x_left_value = math.floor(x_left[y-p0.y])
            x_right_value = math.floor(x_right[y-p0.y])
            for x in range(x_left_value, x_right_value+1):
                canvas.put_pixel(x,y,color,line_width)
                
    def draw_shaded_triangle(self, p0, p1, p2, color, line_width):
        # Sort the points so that y0 <= y1 <= y2
        if p1.y < p0.y:
            p1, p0 = p0, p1
        if p2.y < p0.y:
            p2, p0 = p0, p2
        if p2.y < p1.y:
            p1, p2 = p2, p1
            
        # Compute the x coordinates and h values of the triangle edges
        x01 = self.interpolate(p0.y, p0.x, p1.y, p1.x)
        h01 = self.interpolate(p0.y, p0.h, p1.y, p1.h)
        
        x12 = self.interpolate(p1.y, p1.x, p2.y, p2.x)
        h12 = self.interpolate(p1.y, p1.h, p2.y, p2.h)
        
        x02 = self.interpolate(p0.y, p0.x, p2.y, p2.x)
        h02 = self.interpolate(p0.y, p0.h, p2.y, p2.h)
        
        # Concatenate the short sides
        x01.pop(-1)
        x012 = x01 + x12
        
        h01.pop(-1)
        h012 = h01 + h12
        
        # Determine which is left and which is right
        m = math.floor(len(x012)/2)
        if x02[m] < x012[m]:
            x_left = x02
            h_left = h02
            
            x_right= x012
            h_right = h012
        else:
            x_left = x012
            h_left = h012
            
            x_right = x02
            h_right = h02
            
        # Draw the horizontal segments
        for y in range(p0.y,p2.y):
            x_l= math.floor(x_left[y-p0.y])
            x_r = math.floor(x_right[y-p0.y])
            
            h_segment = self.interpolate(x_l, h_left[y - p0.y], x_r, h_right[y - p0.y])
            for x in range(x_l, x_r + 1):
                shaded_color = np.dot(color, h_segment[x - x_l])
                canvas.put_pixel(x,y,shaded_color,line_width)
                
    def viewport_to_canvas(self, x, y):
        p = Point(int(x * (canvas.width / camera.viewport_size)), int(y * (canvas.height / camera.viewport_size)), h =1.0)
        return p
    
    def project_vertex(self, v):
        d = camera.projection_plane_z
        return self.viewport_to_canvas(v[0] * d / v[2], v[1] * d / v[2])
    
    def render_triangle(self, triangle, projected):
        
        self.draw_wire_triangle(projected[triangle.v[0]],
                           projected[triangle.v[1]],
                           projected[triangle.v[2]],
                           triangle.color)
        
    def render_model(self, model, transform):
        projected = []
        for vertex in model.vertices:
            vertex_h = Vertex4(vertex.x, vertex.y, vertex.z, 1)
            projected.append(self.project_vertex(np.dot(transform, np.array([vertex_h.x, vertex_h.y, vertex_h.z, vertex_h.w]))))
            
        for triangle in triangles:
            self.render_triangle(triangle, projected)
            
    def render_scene(self, instances):
        camera_matrix = np.dot(rotation_matrix(self.camera.orientation).T, translate_matrix(np.dot(-1, self.camera.position)))
        
        for instance in instances:
            transform = np.dot(camera_matrix, instance.transform)
            self.render_model(instance.model, transform)
            
    def render(self):
        plt.imshow(self.canvas.img)
        return
    
    def clip_scene(self, scene, planes):
        clipped_instances = []
        for i in scene:
            clipped_instance = self.clip_instance(i, planes)
            if clipped_instance != None:
                clipped_instances.append(clipped_instance)

        clipped_scene = Copy(scene)
        clipped_scene.instances = clipped_instances

        return clipped_scene

    def clip_instance(self, instance, planes):
        for p in planes:
            instance = clip_instance_against_plane(instance, p)
            if instance == None:
                return None

        return instance

    def clip_instance_against_plane(self, instance, plane):
        d = signed_distance(plane, instance.bounding_sphere.center)
        if d > r :
            return instance

        elif d < -r :
            return None

        else:
            clipped_instance = Copy(instance)
            clipped_instance.triangles = clip_triangles_against_plane(instance.triangles, plane)

            return clipped_instance

    def clip_triangles_against_plane(self, triangles, plane):
        clipped_triangles = []
        for t in triangles:
            clipped_triangles.append(clip_triangle(t, plane))

        return clipped_trianles
    
    def intersection(line, plane):
        
    
    def clip_triangle(self, triangle, plane):
        d0 = self.signed_distance(plane, triangle.v0)
        d1 = self.signed_distance(plane, triangle.v1)
        d2 = self.signed_distance(plane, triangle.v2)

        if d0 >= 0 and d1 >= 0 and d2 >= 0:
            return triangle
        
        elif d0 < 0 and d1 < 0 and d2 < 0:
            return []
        
        elif d0 * d1 * d2 > 0:
            if d0 > 0:
                a = d0
                b_ = intersection(ab, plane)
                c_ = intersection(ac, plane)
                return triangle(a, b_, c_)
                
            elif d1 > 0:
                a = d1
                b_ = intersection(ab, plane)
                c_ = intersection(ac, plane)
                return triangle(a, b_, c_)
            
            else:
                a = d2
                b_ = intersection(ab, plane)
                c_ = intersection(ac, plane)
                return triangle(a, b_, c_)
            
        else:
            if d0 < 0:
                c = d0
                a = d1
                b = d2
                
                a_ = intersection(ac, plane)
                b_ = intersection(bc, plane)
                
                return (triangle(c,a_,b_))
            elif d1 < 0:
                c = d1
                a = d0
                b = d2
                
                a_ = intersection(ac, plane)
                b_ = intersection(bc, plane)
                
                return (triangle(c,a_,b_))
            elif d2 < 0:
                c = d2
                a = d1
                b = d0
                
                a_ = intersection(ac, plane)
                b_ = intersection(bc, plane)
                
                return (triangle(c,a_,b_))
            

    def signed_distance(self, plane, vertex):
        normal = plane.normal
        
        return (vertex.x * normal[0]) + (vertex.y * normal[1]) + (vertex.z * normal[2]) + plane.d
    


In [31]:
camera = Camera([-3,1,2], [0,-30,0], viewport_size = 1, projection_plane_z = 1)
canvas = Canvas(width=600, height=600)

renderer = Renderer(camera=camera, canvas=canvas)

vertices = [
    Vertex(1,1,1),
    Vertex(-1,1,1),
    Vertex(-1,-1,1),
    Vertex(1,-1,1),
    Vertex(1,1,-1),
    Vertex(-1,1,-1),
    Vertex(-1,-1,-1),
    Vertex(1,-1,-1)
]

triangles = [
    Triangle(0,1,2,(255,0,0)),
    Triangle(0,2,3,(255,0,0)),
    Triangle(4,0,3,(0,255,0)),
    Triangle(4,3,7,(0,255,0)),
    Triangle(5,4,7,(0,0,255)),
    Triangle(5,7,6,(0,0,255)),
    Triangle(1,5,6,(255,255,0)),
    Triangle(1,6,2,(255,255,0)),
    Triangle(4,5,1,(255,0,255)),
    Triangle(4,1,0,(255,0,255)),
    Triangle(2,6,7,(0,255,255)),
    Triangle(2,7,3,(0,255,255))
]

cube = Model(vertices, triangles)

instances = [Instance(cube, [-1.5, 0, 7], [0,0,0], 0.75),
            Instance(cube, [1.25, 2.5, 7.5], [0,195,0])]

renderer.clip_scene(instances,camera.planes)
renderer.render()

NameError: name 'signed_distance' is not defined